In [143]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [144]:
try:
    print('Loading local files:')
    features = pd.read_parquet('../data/features.parquet')
    target = pd.read_parquet('../data/target.parquet')
except:
    print('Getting data and models from IPFS:')
    ! cd .. && ./get_data.sh

Loading local files:


In [138]:
dataset = pd.concat([features, target], axis=1)

In [139]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [140]:
train_features = train_dataset.iloc[:, :-1]
test_features = test_dataset.iloc[:, :-1]

In [141]:
train_target = train_features.iloc[:, -1]
test_target = test_features.iloc[:, -1]

In [146]:
try:
    model = tf.keras.models.load_model('../data/model')
    # Check its architecture
    model.summary()
except:
    model = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(1)
    ])
    
    model.compile(loss='mean_squared_error',
            optimizer=tf.keras.optimizers.Adam(0.001))
    
    history = model.fit(
    train_features,
    train_target,
    validation_split=0.2,
    verbose=True, epochs=100)
    
    def plot_loss(history):
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.xlabel('Epoch')
        plt.ylabel('Error [MPG]')
        plt.legend()
        plt.grid(True)

    plot_loss(history)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 4)                 68        
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 9,641
Trainable params: 9,641
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.evaluate(
    test_features,
    test_target, verbose=True)

1600/1600 [==============================] - 1s 695us/step - loss: 6.4790e-06


6.47903470962774e-06

In [112]:
y_hat = model.predict(test_features)

1600/1600 [==============================] - 1s 686us/step


In [113]:
import scipy
scipy.stats.pearsonr(np.ravel(y_hat), test_target)

(0.9999924268520269, 0.0)

In [145]:
model.save('../data/model')

INFO:tensorflow:Assets written to: ../data/model/assets


INFO:tensorflow:Assets written to: ../data/model/assets
